<a href="https://colab.research.google.com/github/wojiushilr/pytorch_training/blob/master/03_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 引包定义超参数

In [102]:
import time
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [103]:
# 定义超参数
batch_size = 64
learning_rate = 1e-3
num_epochs = 100

### 下载数据集mnist

In [104]:
train_dataset = datasets.FashionMNIST(
    root='../datasets', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.FashionMNIST(
    root='../datasets', train=False, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### 看看数据集里都是什么格式
### 看起来data[0] -> image data
### data[1] -> imgae label

In [105]:
print(type(train_dataset))
print(train_loader)
for i, data in enumerate(train_loader,0):
  print(data[1])
  print(type(data)) ## <class 'list'>
  print(type(data[1])) ## <class 'torch.Tensor'>
  print(len(data[1]))
  print(i)
  break

<class 'torchvision.datasets.mnist.FashionMNIST'>
tensor([5, 9, 9, 7, 4, 2, 3, 2, 7, 2, 4, 3, 7, 8, 3, 1, 3, 6, 7, 1, 6, 8, 2, 7,
        3, 1, 4, 3, 1, 2, 9, 2, 8, 9, 6, 3, 4, 0, 2, 9, 5, 9, 2, 8, 4, 7, 1, 6,
        4, 4, 4, 8, 0, 0, 2, 8, 7, 6, 3, 0, 3, 5, 4, 6])
<class 'list'>
<class 'torch.Tensor'>
64
0


### 定义模型， 如果有GPU则使用GPU

In [106]:
# logistic Regression Model
class LogisticRegression(nn.Module):
    def __init__(self,in_dim, n_class):
        super(LogisticRegression, self).__init__()
        self.logistic = nn.Linear(in_dim, n_class)  # input and output is 1 dimension

    def forward(self, x):
        out = self.logistic(x)
        return out
  

model = LogisticRegression(28*28 ,10)
if torch.cuda.is_available():
    model = model.cuda()
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### 训练

In [ ]:
for epoch in range(num_epochs):
  starttime = time.time()
  train_acc = 0.0
  train_loss = 0.0
  val_acc = 0.0
  val_loss = 0.0
  for i, data in enumerate(train_loader):
    img, label = data
    img = img.view(img.size(0), -1)  # 将图片展开成 28x28
    if torch.cuda.is_available():
      img = img.cuda()
      label = label.cuda()
    # 向前传播
    train_pred = model(img)
    batch_loss = loss(train_pred, label)
    train_loss += batch_loss.item()
    _, pred = torch.max(train_pred, 1)
    train_acc += (pred==label).float().mean()
    optimizer.zero_grad()
    batch_loss.backward() # 算出每個參數的 gradient
    optimizer.step() # 以 optimizer 用 gradient 更新參數值
  if i % 300 == 0:
    # print(f'[{epoch+1}/{num_epochs}] Loss: {train_loss/i:.6f}, Acc: {train_acc　/i:.6f}')
    print("{} / {} Loss: {}, Acc: {}".format(epoch+1,num_epochs, train_loss/i, train_acc/i))
  print("{} / {} Loss: {}, Acc: {}".format(epoch+1, num_epochs, train_loss/i, train_acc/i))
    
  model.eval()
  for data in test_loader:
      img, label = data
      img = img.view(img.size(0), -1)
      if torch.cuda.is_available():
          img = img.cuda()
          label = label.cuda()
      with torch.no_grad():
          val_pred = model(img)
          batch_loss = loss(val_pred, label)
      val_loss += batch_loss.item()
      _, pred = torch.max(val_pred, 1)
      val_acc += (pred == label).float().mean()
  print(f'Test Loss: {val_loss/len(test_loader):.6f}, Acc: {val_acc/len(test_loader):.6f}')
  print(f'Time:{(time.time()-starttime):.1f} s')

# 保存模型
torch.save(model.state_dict(), './logstic.pth')


## Some tip about axis
## https://www.jianshu.com/p/64a7fb9ac310

In [ ]:
a = torch.tensor([[1,3,7],[4,5,6]])
print(torch.max(a,0))
print(a.max())

In [90]:
print("{} / {} Loss: {}, Acc: {}".format("1","1","1","1"))
print("{} / {} Loss: {}, Acc: {}".format(100+1,222, 22/2, 222/2))

1 / 1 Loss: 1, Acc: 1
101 / 222 Loss: 11.0, Acc: 111.0


### 小例子解释本Script是怎么计算accuracy的

In [116]:
train_acc = 0
pred = torch.tensor([1,2,3,4]) 
actual = torch.tensor([1,2,2,4]) 
train_acc = (pred==actual) # tensor([ True,  True, False,  True])
print(train_acc)
train_acc = (pred==actual).float() # tensor([1., 1., 0., 1.])
print(train_acc)
train_acc = (pred==actual).float().mean() # tensor(0.7500)
print(train_acc)

tensor([ True,  True, False,  True])
tensor([1., 1., 0., 1.])
tensor(0.7500)
